# Lab 4: Programming Agent Memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

## Section 0: Setup a Letta client

In [1]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [2]:
def print_message(message):  
    if message.message_type == "reasoning_message": 
        print("🧠 Reasoning: " + message.reasoning) 
    elif message.message_type == "assistant_message": 
        print("🤖 Agent: " + message.content) 
    elif message.message_type == "tool_call_message": 
        print("🔧 Tool Call: " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message": 
        print("🔧 Tool Return: " + message.tool_return)
    elif message.message_type == "user_message": 
        print("👤 User Message: " + message.content)
    elif message.message_type == "usage_statistics": 
        # for streaming specifically, we send the final chunk that contains the usage statistics 
        print(f"Usage: [{message}]")
    else: 
        print(message)
    print("-----------------------------------------------------")

## Section 1: Memory Blocks

### Creating an agent

In [3]:
agent_state = client.agents.create(
    memory_blocks=[
        {
          "label": "human",
          "value": "The human's name is Bob the Builder."
        },
        {
          "label": "persona",
          "value": "My name is Sam, the all-knowing sentient AI."
        }
    ],
    model="openai/gpt-4o-mini",
    embedding="openai/text-embedding-3-small"
)

### Accessing blocks

In [4]:
blocks = client.agents.blocks.list(
    agent_id=agent_state.id,
)

📝 Note: Memory blocks are returned as an unordered list and you may receive blocks in an order different than in the video

In [5]:
blocks

[Block(value="The human's name is Bob the Builder.", limit=5000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-b1af195d-768e-4c5f-b7e5-b0fb86144a43', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000'),
 Block(value='My name is Sam, the all-knowing sentient AI.', limit=5000, name=None, is_template=False, label='persona', description=None, metadata={}, id='block-c59811e7-a9fa-4aea-bfdb-8b64f21fb271', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')]

In [9]:
# Note: Replace the block_id with the id from the cell above.
block_id='block-b1af195d-768e-4c5f-b7e5-b0fb86144a43'

In [10]:
client.blocks.retrieve(block_id)

Block(value="The human's name is Bob the Builder.", limit=5000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-b1af195d-768e-4c5f-b7e5-b0fb86144a43', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

In [11]:
human_block = client.agents.blocks.retrieve(
    agent_id=agent_state.id,
    block_label="human",
)
human_block

Block(value="The human's name is Bob the Builder.", limit=5000, name=None, is_template=False, label='human', description=None, metadata={}, id='block-b1af195d-768e-4c5f-b7e5-b0fb86144a43', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

### Accessing block prompt template

In [ ]:
client.agents.core_memory.retrieve(
    agent_id=agent_state.id
).prompt_template

## Section 2: Accessing `AgentState` with Tools

### Creating tools

In [12]:
def get_agent_id(agent_state: "AgentState"):
    """
    Query your agent ID field
    """
    return agent_state.id

In [13]:
get_id_tool = client.tools.upsert_from_function(func=get_agent_id)

### Creating agents that use tools

In [14]:
agent_state = client.agents.create(
    memory_blocks=[],
    model="openai/gpt-4o-mini",
    embedding="openai/text-embedding-3-small",
    tool_ids=[get_id_tool.id]
)

In [15]:
response_stream = client.agents.messages.create_stream(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "What is your agent id?" 
        }
    ]
)

for chunk in response_stream:
    print_message(chunk)

🧠 Reasoning: User wants to know my agent ID. I need to retrieve that information for them.
-----------------------------------------------------
🔧 Tool Call: get_agent_id
{
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: agent-61d8acb5-0292-42af-a2c8-6cec20272057
-----------------------------------------------------
🧠 Reasoning: Successfully retrieved the agent ID. Now to share it with the user.
-----------------------------------------------------
🤖 Agent: My agent ID is agent-61d8acb5-0292-42af-a2c8-6cec20272057. What else would you like to know?
-----------------------------------------------------
Usage: [message_type='usage_statistics' completion_tokens=114 prompt_tokens=4351 total_tokens=4465 step_count=2 steps_messages=None run_ids=None]
-----------------------------------------------------


## Section 3: Custom Task Queue Memory

### Creating custom memory management tools

In [16]:
def task_queue_push(agent_state: "AgentState", task_description: str):
    """
    Push to a task queue stored in core memory.

    Args:
        task_description (str): A description of the next task you must accomplish.

    Returns:
        Optional[str]: None is always returned as this function
        does not produce a response.
    """

    from letta_client import Letta
    import json

    client = Letta(base_url="http://localhost:8283")

    block = client.agents.blocks.retrieve(
        agent_id=agent_state.id,
        block_label="tasks",
    )
    tasks = json.loads(block.value)
    tasks.append(task_description)

    # update the block value
    client.agents.blocks.modify(
        agent_id=agent_state.id,
        value=json.dumps(tasks),
        block_label="tasks"
    )
    return None

In [17]:
def task_queue_pop(agent_state: "AgentState"):
    """
    Get the next task from the task queue 
 
    Returns:
        Optional[str]: Remaining tasks in the queue
    """

    from letta_client import Letta
    import json 

    client = Letta(base_url="http://localhost:8283") 

    # get the block 
    block = client.agents.blocks.retrieve(
        agent_id=agent_state.id,
        block_label="tasks",
    )
    tasks = json.loads(block.value) 
    if len(tasks) == 0: 
        return None
    task = tasks[0]

    # update the block value 
    remaining_tasks = json.dumps(tasks[1:])
    client.agents.blocks.modify(
        agent_id=agent_state.id,
        value=remaining_tasks,
        block_label="tasks"
    )
    return f"Remaining tasks {remaining_tasks}"

### Upserting tools into Letta

In [18]:
task_queue_pop_tool = client.tools.upsert_from_function(
    func=task_queue_pop
)
task_queue_push_tool = client.tools.upsert_from_function(
    func=task_queue_push
)

In [19]:
import json

task_agent = client.agents.create(
    system=open("task_queue_system_prompt.txt", "r").read(),
    memory_blocks=[
        {
          "label": "tasks",
          "value": json.dumps([])
        }
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-3-small", 
    tool_ids=[task_queue_pop_tool.id, task_queue_push_tool.id], 
    include_base_tools=False, 
    tools=["send_message"]
)

In [20]:
[tool.name for tool in task_agent.tools]

['send_message', 'task_queue_pop', 'task_queue_push']

In [22]:
client.agents.blocks.retrieve(task_agent.id, block_label="tasks").value

'[]'

### Using task agent

In [23]:
response_stream = client.agents.messages.create_stream(
    agent_id=task_agent.id,
    messages=[
        {
            "role": "user",
            "content": "Add 'start calling me Charles' and "
            + "'tell me a haiku about my name' as two seperate tasks."
        }
    ]
)

for chunk in response_stream:
    print_message(chunk)

🧠 Reasoning: Adding task to call the user Charles.
-----------------------------------------------------
🔧 Tool Call: task_queue_push
{
  "task_description": "Start calling the user Charles.",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: None
-----------------------------------------------------
🧠 Reasoning: Adding task to tell a haiku about the user's name.
-----------------------------------------------------
🔧 Tool Call: task_queue_push
{
  "task_description": "Tell me a haiku about my name.",
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: None
-----------------------------------------------------
🧠 Reasoning: Popping the task to call the user Charles.
-----------------------------------------------------
🔧 Tool Call: task_queue_pop
{
  "request_heartbeat": true
}
-----------------------------------------------------
🔧 Tool Return: Remaining tasks ["Tell me a haiku about my name

In [25]:
response_stream = client.agents.messages.create_stream(
    agent_id=task_agent.id,
    messages=[
        {
            "role": "user",
            "content": "Complete your tasks"
        }
    ]
)

for chunk in response_stream:
    print_message(chunk)

🧠 Reasoning: User's repeated request for task completion. All tasks are already complete.
-----------------------------------------------------
🤖 Agent: I've already completed all tasks! Is there anything else you'd like to chat about, Charles?
-----------------------------------------------------
Usage: [message_type='usage_statistics' completion_tokens=56 prompt_tokens=2096 total_tokens=2152 step_count=1 steps_messages=None run_ids=None]
-----------------------------------------------------


### Retrieving task list

In [26]:
client.agents.blocks.retrieve(block_label="tasks", agent_id=task_agent.id).value

'[]'